<a href="https://colab.research.google.com/github/aggelinacha/pytorch_challenge/blob/project/Image_Classifier_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Developing an AI application

Going forward, AI algorithms will be incorporated into more and more everyday applications. For example, you might want to include an image classifier in a smart phone app. To do this, you'd use a deep learning model trained on hundreds of thousands of images as part of the overall application architecture. A large part of software development in the future will be using these types of models as common parts of applications. 

In this project, you'll train an image classifier to recognize different species of flowers. You can imagine using something like this in a phone app that tells you the name of the flower your camera is looking at. In practice you'd train this classifier, then export it for use in your application. We'll be using [this dataset](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html) of 102 flower categories, you can see a few examples below. 

<img src='assets/Flowers.png' width=500px>

The project is broken down into multiple steps:

* Load and preprocess the image dataset
* Train the image classifier on your dataset
* Use the trained classifier to predict image content

We'll lead you through each part which you'll implement in Python.

When you've completed this project, you'll have an application that can be trained on any set of labeled images. Here your network will be learning about flowers and end up as a command line application. But, what you do with your new skills depends on your imagination and effort in building a dataset. For example, imagine an app where you take a picture of a car, it tells you what the make and model is, then looks up information about it. Go build your own dataset and make something new.

First up is importing the packages you'll need. It's good practice to keep all the imports at the beginning of your code. As you work through this notebook and find you need to import a package, make sure to add the import up here.

In [0]:
!pip install pillow==4.1.1
!pip install torch==1.0.0
!pip install torchvision==0.2.1



    100% |████████████████████████████████| 5.7MB 1.2MB/s 
  Found existing installation: Pillow 4.0.0
    Uninstalling Pillow-4.0.0:
      Successfully uninstalled Pillow-4.0.0
    100% |████████████████████████████████| 591.8MB 27kB/s 
tcmalloc: large alloc 1073750016 bytes == 0x61d7e000 @  0x7f8987db22a4 0x591a07 0x5b5d56 0x502e9a 0x506859 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x504c28 0x502540 0x502f3d 0x507641
    100% |████████████████████████████████| 61kB 4.8MB/s 


In [0]:
# Imports here
import torch
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from PIL import Image
from collections import OrderedDict
import os
import json

## Load the data

Here you'll use `torchvision` to load the data ([documentation](http://pytorch.org/docs/0.3.0/torchvision/index.html)). You can [download the data here](https://s3.amazonaws.com/content.udacity-data.com/courses/nd188/flower_data.zip). The dataset is split into two parts, training and validation. For the training, you'll want to apply transformations such as random scaling, cropping, and flipping. This will help the network generalize leading to better performance. If you use a pre-trained network, you'll also need to make sure the input data is resized to 224x224 pixels as required by the networks.

The validation set is used to measure the model's performance on data it hasn't seen yet. For this you don't want any scaling or rotation transformations, but you'll need to resize then crop the images to the appropriate size.

The pre-trained networks available from `torchvision` were trained on the ImageNet dataset where each color channel was normalized separately. For both sets you'll need to normalize the means and standard deviations of the images to what the network expects. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`, calculated from the ImageNet images.  These values will shift each color channel to be centered at 0 and range from -1 to 1.

In [0]:
def load_data(data_dir, batch_size=20, n_workers=0):
    """
    @brief Load train and validation data and apply transformations.

    @param batch_size (\a int) How many samples per batch to load.
    @param num_workers (\a int) Number of subprocesses to use for data
        loading.

    @returns \b train_loader DataLoader for train data.
    @returns \b valid_loader DataLoader for validation data.
    """
    train_dir = os.path.join(data_dir, 'train')
    valid_dir = os.path.join(data_dir, 'valid')

    # Define transforms
    train_transforms = transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.RandomRotation(10),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406),
                             (0.229, 0.224, 0.225))])
    val_transforms = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406),
                             (0.229, 0.224, 0.225))])

    # Load the datasets with ImageFolder
    train_data = datasets.ImageFolder(train_dir,
                                      transform=train_transforms)
    val_data = datasets.ImageFolder(valid_dir,
                                    transform=val_transforms)

    # Define the dataloaders
    train_loader = torch.utils.data.DataLoader(train_data,
                                               batch_size=batch_size,
                                               num_workers=n_workers,
                                               shuffle=True)
    valid_loader = torch.utils.data.DataLoader(val_data,
                                               batch_size=batch_size,
                                               num_workers=n_workers,
                                               shuffle=True)

    return train_loader, valid_loader, train_data.class_to_idx

### Label mapping

You'll also need to load in a mapping from category label to category name. You can find this in the file `cat_to_name.json`. It's a JSON object which you can read in with the [`json` module](https://docs.python.org/2/library/json.html). This will give you a dictionary mapping the integer encoded categories to the actual names of the flowers.

In [0]:
def get_label_mapping(input_json="cat_to_name.json"):
    """!
    @brief Read json file with label mapping (from category label to
        category name).
    """
    with open(input_json, "r") as f:
        cat_to_name = json.load(f)

    return cat_to_name


# Building and training the classifier

Now that the data is ready, it's time to build and train the classifier. As usual, you should use one of the pretrained models from `torchvision.models` to get the image features. Build and train a new feed-forward classifier using those features.

We're going to leave this part up to you. If you want to talk through it with someone, chat with your fellow students! You can also ask questions on the forums or join the instructors in office hours.

Refer to [the rubric](https://review.udacity.com/#!/rubrics/1663/view) for guidance on successfully completing this section. Things you'll need to do:

* Load a [pre-trained network](http://pytorch.org/docs/master/torchvision/models.html) (If you need a starting point, the VGG networks work great and are straightforward to use)
* Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
* Train the classifier layers using backpropagation using the pre-trained network to get the features
* Track the loss and accuracy on the validation set to determine the best hyperparameters

We've left a cell open for you below, but use as many as you need. Our advice is to break the problem up into smaller parts you can run separately. Check that each part is doing what you expect, then move on to the next. You'll likely find that as you work through each part, you'll need to go back and modify your previous code. This is totally normal!

When training make sure you're updating only the weights of the feed-forward network. You should be able to get the validation accuracy above 70% if you build everything right. Make sure to try different hyperparameters (learning rate, units in the classifier, epochs, etc) to find the best model. Save those hyperparameters to use as default values in the next part of the project.

In [0]:
# check if CUDA is available
def check_cuda():
    """!
    @brief Check if CUDA is available.
    """
    train_on_gpu = torch.cuda.is_available()

    if not train_on_gpu:
        print('CUDA is not available. Using CPU ...')
    else:
        print('CUDA is available! Using GPU ...')

    return train_on_gpu

train_on_gpu = check_cuda()

CUDA is available! Using GPU ...


In [0]:
def build_model(n_classes, pretrained="vgg16", train_on_gpu=False):
    """!
    @brief Build network architecture using a pretrained model.
           (Available choices: vgg16, vgg19, densenet161 and resnet152)
    """
    if pretrained[:3] == "vgg":
        model = build_model_vgg(n_classes, pretrained=pretrained)
    elif pretrained[:8] == "densenet":
        model = build_model_densenet(n_classes, pretrained=pretrained)
    elif pretrained[:6] == "resnet":
        model = build_model_resnet(n_classes, pretrained=pretrained)

    # if GPU is available, move the model to GPU
    if train_on_gpu:
        model.cuda()

#     print(model)

    return model


def build_model_densenet(n_classes, pretrained="densenet161"):
    """!
    @brief Build network architecture using a pretrained DenseNet model.
    """
    if pretrained == "densenet161":
        model = models.densenet161(pretrained=True)
    else:
        raise NotImplementedError("Use of {} as pretrained network for "
                                  "feature extraction is not "
                                  "implemented yet".format(pretrained))

    # Freeze training for all features layers
    for param in model.features.parameters():
        param.requires_grad = False

    # Last linear layer
    n_inputs = model.classifier.in_features
    model.classifier = nn.Linear(n_inputs, n_classes)

    return model


def build_model_resnet(n_classes, pretrained="resnet152"):
    """!
    @brief Build network architecture using a pretrained ResNet model.
    """
    if pretrained == "resnet152":
        model = models.resnet152(pretrained=True)
    else:
        raise NotImplementedError("Use of {} as pretrained network for "
                                  "feature extraction is not "
                                  "implemented yet".format(pretrained))

    # Freeze training for all features layers
    for param in model.parameters():
        param.requires_grad = True

    # Last linear layer
    n_inputs = model.fc.in_features
    model.fc = nn.Linear(n_inputs, n_classes)

    return model


def build_model_vgg(n_classes, pretrained="vgg16"):
    """!
    @brief Build network architecture using a pretrained VGG model.
    """
    if pretrained == "vgg16":
        model = models.vgg16(pretrained=True)
    elif pretrained == "vgg19":
        model = models.vgg19(pretrained=True)
    else:
        raise NotImplementedError("Use of {} as pretrained network for "
                                  "feature extraction is not "
                                  "implemented yet".format(pretrained))

    # Freeze training for all features layers
    for param in model.features.parameters():
        param.requires_grad = False

    # Last linear layer
    n_inputs = model.classifier[6].in_features
    model.classifier[6] = nn.Linear(n_inputs, n_classes)

    return model

In [0]:
def train_model(model, train_loader, criterion, optimizer,
                train_on_gpu=False):
    """!
    @brief Train model.
    """
    train_loss = 0.0
    model.train()

    for batch_idx, (data, target) in enumerate(train_loader):
        # move tensors to GPU if CUDA is available
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass
        # compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the batch loss
        loss = criterion(output, target)
        # backward pass
        # compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update training loss
        train_loss += loss.item() * data.size(0)

    train_loss = train_loss / len(train_loader.dataset)

    return train_loss

In [0]:
def evaluate_model(model, valid_loader, criterion, n_classes=10,
                   train_on_gpu=False):
    """!
    @brief Validate model.
    """
    valid_loss = 0.0
    class_correct = list(0. for i in range(n_classes))
    class_total = list(0. for i in range(n_classes))

    model.eval()

    for batch_idx, (data, target) in enumerate(valid_loader):
        # move tensors to GPU if CUDA is available
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
        # forward pass
        # compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the batch loss
        loss = criterion(output, target)
        # update average validation loss
        valid_loss += loss.item() * data.size(0)
        # convert output probabilities to predicted class
        _, pred = torch.max(output, 1)
        # compare predictions to true label
        correct_tensor = pred.eq(target.data.view_as(pred))
        if not train_on_gpu:
            correct = np.squeeze(correct_tensor.numpy())
        else:
            correct = np.squeeze(correct_tensor.cpu().numpy())
        # calculate test accuracy for each object class
        for i in range(target.size(0)):
            label = target.data[i]
            class_correct[label] += correct[i].item()
            class_total[label] += 1

    valid_loss = valid_loss / len(valid_loader.dataset)
    valid_acc = 100. * np.sum(class_correct) / np.sum(class_total)

    return valid_loss, valid_acc

In [0]:
def save_checkpoint(outfile, model, optimizer, loss, epoch,
                    class_to_idx, pretrained="vgg16"):
    """!
    @brief Save model checkpoint.
    """
    checkpoint = {'model': model.state_dict(),
                  'optimizer_state': optimizer.state_dict(),
                  'loss': loss,
                  'epoch': epoch,
                  'class_to_idx': class_to_idx,
                  'pretrained': pretrained}

    torch.save(checkpoint, outfile)

In [0]:
def load_checkpoint(filepath, train_on_gpu=False):
    """!
    @brief Load model checkpoint.
    """
    loc = 'cuda' if train_on_gpu else 'cpu'
    checkpoint = torch.load(filepath, map_location=loc)

    label2idx = checkpoint['class_to_idx']
    num_classes = len(label2idx)

    model = build_model(num_classes,
                        pretrained=checkpoint['pretrained'],
                        train_on_gpu=train_on_gpu)
    model.load_state_dict(checkpoint['model'], strict=False)
    if train_on_gpu:
        model.cuda()

    return model, checkpoint


In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

loader_tr, loader_val, class2idx = \
        load_data("gdrive/My Drive/flower_data",
                  batch_size=16)
num_classes = len(class2idx)

checkpoint = "gdrive/My Drive/checkpoint.pt"
pretrained = "resnet152"

if checkpoint:
    print("Loading model checkpoint...")
    model, ckpt_dict = load_checkpoint(checkpoint,
                                       train_on_gpu=train_on_gpu)
    loss_val_min = ckpt_dict['loss']
    epoch1 = ckpt_dict['epoch'] + 1
    pretrained = ckpt_dict['pretrained']
    class2idx = ckpt_dict['class_to_idx']
else:
    epoch1 = 0
    # Build model architecture
    model = build_model(num_classes,
                        pretrained=pretrained,
                        train_on_gpu=train_on_gpu)
    loss_val_min = np.Inf

# specify loss function (categorical cross-entropy)
criterion = nn.CrossEntropyLoss()

# specify optimizer (stochastic gradient descent) and learning rate
lr = 0.0001
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive
Loading model checkpoint...


Downloading: "https://download.pytorch.org/models/resnet152-b121ed2d.pth" to /root/.torch/models/resnet152-b121ed2d.pth
100%|██████████| 241530880/241530880 [00:02<00:00, 87586304.11it/s]


In [0]:
# Train model
n_epochs = 200
output = "gdrive/My Drive/checkpoint.pt"
for epoch in range(epoch1, n_epochs):
    loss_tr = train_model(model,
                          loader_tr,
                          criterion,
                          optimizer,
                          train_on_gpu=train_on_gpu)
    loss_val, acc_val = evaluate_model(model,
                                       loader_val,
                                       criterion,
                                       n_classes=num_classes,
                                       train_on_gpu=train_on_gpu)
    # Print training/validation statistics
    print('Epoch: {} \tTraining Loss: {:.6f} '
          '\tValidation Loss: {:.6f}'
          '\tValidation Accuracy: {:.2f}'
          .format(epoch + 1,
                  loss_tr,
                  loss_val,
                  acc_val))

    # Save model if validation loss has decreased
    if loss_val <= loss_val_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}). '
              'Saving model ...'.format(loss_val_min,
                                        loss_val))
        save_checkpoint(output,
                        model,
                        optimizer,
                        loss_val,
                        epoch,
                        class2idx,
                        pretrained=pretrained)
        loss_val_min = loss_val

Epoch: 141 	Training Loss: 0.079595 	Validation Loss: 0.059975	Validation Accuracy: 98.90
Epoch: 142 	Training Loss: 0.078050 	Validation Loss: 0.063575	Validation Accuracy: 98.66
Epoch: 143 	Training Loss: 0.086386 	Validation Loss: 0.058702	Validation Accuracy: 99.02


## Save the checkpoint

Now that your network is trained, save the model so you can load it later for making predictions. You probably want to save other things such as the mapping of classes to indices which you get from one of the image datasets: `image_datasets['train'].class_to_idx`. You can attach this to the model as an attribute which makes inference easier later on.

```model.class_to_idx = image_datasets['train'].class_to_idx```

Remember that you'll want to completely rebuild the model later so you can use it for inference. Make sure to include any information you need in the checkpoint. If you want to load the model and keep training, you'll want to save the number of epochs as well as the optimizer state, `optimizer.state_dict`. You'll likely want to use this trained model in the next part of the project, so best to save it now.

## Loading the checkpoint

At this point it's good to write a function that can load a checkpoint and rebuild the model. That way you can come back to this project and keep working on it without having to retrain the network.

# Inference for classification

Now you'll write a function to use a trained network for inference. That is, you'll pass an image into the network and predict the class of the flower in the image. Write a function called `predict` that takes an image and a model, then returns the top $K$ most likely classes along with the probabilities. It should look like 

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

First you'll need to handle processing the input image such that it can be used in your network. 

## Image Preprocessing

You'll want to use `PIL` to load the image ([documentation](https://pillow.readthedocs.io/en/latest/reference/Image.html)). It's best to write a function that preprocesses the image so it can be used as input for the model. This function should process the images in the same manner used for training. 

First, resize the images where the shortest side is 256 pixels, keeping the aspect ratio. This can be done with the [`thumbnail`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) or [`resize`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) methods. Then you'll need to crop out the center 224x224 portion of the image.

Color channels of images are typically encoded as integers 0-255, but the model expected floats 0-1. You'll need to convert the values. It's easiest with a Numpy array, which you can get from a PIL image like so `np_image = np.array(pil_image)`.

As before, the network expects the images to be normalized in a specific way. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`. You'll want to subtract the means from each color channel, then divide by the standard deviation. 

And finally, PyTorch expects the color channel to be the first dimension but it's the third dimension in the PIL image and Numpy array. You can reorder dimensions using [`ndarray.transpose`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ndarray.transpose.html). The color channel needs to be first and retain the order of the other two dimensions.

In [0]:
def process_image(image_path):
    """!
    @brief Scale, crop, and normalize a PIL image for a PyTorch model.
        Returns a Pytorch tensor.

    @param image_path (\a str) Path of the input image to load.

    @returns \b image (\a PyTorch tensor) Pre-processed image.
    """
    size_scale = 256
    size_crop = 224

    # Load PIL image
    pil_image = Image.open(image_path)

    # Resize
    height, width = pil_image.size
    pil_image = pil_image.resize(
        (max(size_scale, int(size_scale * height / width)),
         max(size_scale, int(size_scale * width / height))))

    # Crop
    center_h = pil_image.size[0] // 2
    center_v = pil_image.size[1] // 2
    left = center_h - (size_crop / 2)
    right = center_h + (size_crop / 2)
    upper = center_v - (size_crop / 2)
    lower = center_v + (size_crop / 2)
    pil_image = pil_image.crop(box=(left, upper, right, lower))
    np_image = np.array(pil_image, dtype=float)
    np_image = np_image / 255.0

    # Normalize
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    np_image = (np_image - mean) / std

    # Convert to PyTorch tensor
    return torch.Tensor(np_image.transpose(2, 0, 1))

To check your work, the function below converts a PyTorch tensor and displays it in the notebook. If your `process_image` function works, running the output through this function should return the original image (except for the cropped out portions).

In [0]:
def imshow(image, ax=None, title=None):
    """!
    @brief Imshow for Pytorch tensor.
    """
    if ax is None:
        fig, ax = plt.subplots()

    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.numpy().transpose((1, 2, 0))

    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean

    # Image needs to be clipped between 0 and 1
    image = np.clip(image, 0, 1)

    ax.imshow(image)
    if title:
        ax.set_title(title)

    return ax

## Class Prediction

Once you can get images in the correct format, it's time to write a function for making predictions with your model. A common practice is to predict the top 5 or so (usually called top-$K$) most probable classes. You'll want to calculate the class probabilities then find the $K$ largest values.

To get the top $K$ largest values in a tensor use [`x.topk(k)`](http://pytorch.org/docs/master/torch.html#torch.topk). This method returns both the highest `k` probabilities and the indices of those probabilities corresponding to the classes. You need to convert from these indices to the actual class labels using `class_to_idx` which hopefully you added to the model or from an `ImageFolder` you used to load the data ([see here](#Save-the-checkpoint)). Make sure to invert the dictionary so you get a mapping from index to class as well.

Again, this method should take a path to an image and a model checkpoint, then return the probabilities and classes.

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

In [0]:
def predict(image, model, topk=5):
    """!
    @brief Predict the class (or classes) of an image using a trained
        deep learning model.

    @param image (\a PyTorch tensor) Batch of images.
    @param model (\a Pytorch model) Trained model.

    @returns \b prob_k Probabilities of the top K classes.
    @returns \b ind_k Indeces of the top K classes.
    """
    model.eval()

    # Get model output
    output = model(image)
    prob = torch.nn.Softmax(dim=1)(output).data
    prob_k, ind_k = prob.topk(topk)

    prob_k = prob_k.cpu().numpy().squeeze()
    ind_k = ind_k.cpu().numpy().squeeze()

    return prob_k, ind_k


## Sanity Checking

Now that you can use a trained model for predictions, check to make sure it makes sense. Even if the validation accuracy is high, it's always good to check that there aren't obvious bugs. Use `matplotlib` to plot the probabilities for the top 5 classes as a bar graph, along with the input image. It should look like this:

<img src='assets/inference_example.png' width=300px>

You can convert from the class integer encoding to actual flower names with the `cat_to_name.json` file (should have been loaded earlier in the notebook). To show a PyTorch tensor as an image, use the `imshow` function defined above.

In [0]:
def map_classes(ind, mapping):
    """!
    @brief Map indices to classes.
    """
    classes = [mapping[idx] for idx in ind]
    return classes


In [0]:
def plot_predictions(probs, labels, topk=5, ax=None):
    """!
    @brief Plot probabilities (bar graph) for top-k predicted classes.
    """
    if ax is None:
        fig, ax = plt.subplots()

    ypos = np.arange(topk)
    ax.barh(ypos, probs)
    ax.set_yticks(ypos)
    ax.set_yticklabels(labels)


In [0]:
def print_results(classes, probs):
    """!
    @brief Print top-k predicted classes along with their probabilities.
    """
    print("Predicted class: '{}' \t(p = {})".format(classes[0],
                                                    probs[0]))
    print("")
    print("Top K classes:")
    print("")
    for i, (c, p) in enumerate(zip(classes, probs), 1):
        print("{}: '{}' \t(p = {})".format(i, c, p))

In [0]:
# Load checkpoint
model, ckpt_dict = load_checkpoint("gdrive/My Drive/checkpoint.pt",
                                   train_on_gpu=train_on_gpu)
cat_to_name = get_label_mapping(
    input_json='gdrive/My Drive/flower_data/cat_to_name.json')
idx_to_class = {idx: cat_to_name[c]
                for c, idx in ckpt_dict['class_to_idx'].items()}

# Display an image along with the top 5 classes
top_k = 5

image_path = 'gdrive/My Drive/flower_data/valid/30/image_03464.jpg'

# Pre-process image
image = process_image(image_path)
image = torch.unsqueeze(image, 0)
if train_on_gpu:
    image = image.cuda()

# Get actual label
label = os.path.basename(os.path.dirname(image_path))
label = cat_to_name[label]

# Predictions - top K classes
prob_k, ind_k = predict(image, model, topk=top_k)
classes_k = map_classes(ind_k, idx_to_class)
print("True label: '{}'".format(label))
print("")
print_results(classes_k, prob_k)

# Plot
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(5, 10))
imshow(torch.squeeze(image.cpu()),
       ax=ax1,
       title=label)
plot_predictions(prob_k,
                 classes_k,
                 ax=ax2,
                 topk=top_k)
plt.show()

# Move model inputs to cuda, if GPU available
